# This notebook walks through the steps of cloning a playlist from a "share" link

In [126]:
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import spotipy.util as util
import os

print(spotipy)

<module 'spotipy' from '/Users/ethanwalker/opt/anaconda3/envs/DS/lib/python3.8/site-packages/spotipy/__init__.py'>


In [74]:
# These parameters come from the developer portal for the wakerXD account, 
# app: TryThis. https://developer.spotify.com/dashboard/applications/411ed1fc291749ccaf9523138836a2dd

SPOTIPY_CLIENT_ID='acbdb84970564d238b485bd68b0f85bd'
SPOTIPY_CLIENT_SECRET=''
REDIRECT_URI='http://spotifyagent.com/callback/'

In [61]:
likedSongURI='spotify:track:1yoHvoWGvLBIybMq9s4dJ2' # This is Spotify's resource identifier
likedSongID=likedSongURI.split(':')[-1] # This saves the last bit 
print(likedSongID)

1yoHvoWGvLBIybMq9s4dJ2


In [62]:
# Permission scopes required by this cloning function
# DOCS: https://developer.spotify.com/documentation/general/guides/authorization/scopes/,
# Authorization code flow: https://developer.spotify.com/documentation/general/guides/authorization/code-flow/
scopes = 'user-read-currently-playing user-library-read playlist-modify-public'

In [75]:
# Creates a link that prompts the user to allow access. Click the link and login or allow, then you should be redirected to the redirect_uri
url = """https://accounts.spotify.com/authorize?client_id={client_id}&response_type=code&redirect_uri={uri}&state=ape&scope={scopes}""".format(client_id=SPOTIPY_CLIENT_ID, uri=REDIRECT_URI, scopes='%20'.join(scopes.split(' ')))

print(url)
# When you are redirected, look at the link of the page, if it contains a code= parameter, copy that value for the next step

https://accounts.spotify.com/authorize?client_id=acbdb84970564d238b485bd68b0f85bd&response_type=code&redirect_uri=http://spotifyagent.com/callback/&state=ape&scope=user-read-currently-playing%20user-library-read%20playlist-modify-public


In [ ]:
# Request an access token
import base64
client_id = SPOTIPY_CLIENT_ID
client_secret = SPOTIPY_CLIENT_SECRET
code = 'AQDq_zEAbhGM-uZas72GaKzHLVGzfPTRu3UybLQ8FQCju_nrECelqLDGkvlPcI_DCCbSZd-uguVX_Kl7RyR2DvcleELTi_mDm7qwMkgQStTFgVygiHxpdKdCKUyWhOsExVd26qaIumFE2Rh4Uu0kelx6U_wfQJiXSPIQ8igcXsTGvC5LYhBnyCvvcbvoBB0pbi1k_0kWtVI0ickjGmXQSjcfv8qpo8TyIlu8mzPNlKVcW25nmvVK7tPjRHoD7MUmKKDsvK-aIWO6ALdypA'

client_credentials = client_id+':'+client_secret
print(client_credentials)
h = "Basic {}".format(base64.b64encode(client_credentials.encode('ascii')).decode())
print(h)

In [99]:
payload ={
    "grant_type":"authorization_code",
    "code":code,
    "redirect_uri":REDIRECT_URI
}
headers ={
    "Authorization":h,
    "Content-Type":"application/x-www-form-urlencoded"
}
resp = requests.post('https://accounts.spotify.com/api/token', params=payload, headers=headers)
print(resp)

<Response [200]>


In [ ]:
resp.text

# Now authorize a spotipy client

In [143]:
oAuth = SpotifyOAuth(username='wakerxd', 
                       scope=scopes.split(' '), 
                       client_id=SPOTIPY_CLIENT_ID, 
                       client_secret=SPOTIPY_CLIENT_SECRET, 
                       redirect_uri=REDIRECT_URI,
                    state='ape')

print(oAuth)

In [144]:
sp = spotipy.Spotify(auth_manager=oAuth)
auth_code = sp.auth_manager.get_auth_response()
print(auth_code)

Enter the URL you were redirected to: http://spotifyagent.com/callback/?code=AQAgerAA8Dfggax9odVmTTUH5wq2LcWMkDpJkRK2HZMv9jb-L0foHjhnwdpCbnz4qs7bNWM7TlW50rak2LDoGw37gLLzVoxK2-5HIwE-IuGIbkz6GSJAbTwp5RiWddVZQOTmhQBB1PaWMpDOuJ4N7mbacOVuD6BZvj-B14I5bKF3P4xhrt6fPHFSrFBOIeNN4715cEbToWRECILbPRZzFzHMzpHLFtUCidHS-qOphkIkrG6CzYPMsxAYIcPGCQPOVvKfUQhyfKfqz4c0aQ&state=ape
AQAgerAA8Dfggax9odVmTTUH5wq2LcWMkDpJkRK2HZMv9jb-L0foHjhnwdpCbnz4qs7bNWM7TlW50rak2LDoGw37gLLzVoxK2-5HIwE-IuGIbkz6GSJAbTwp5RiWddVZQOTmhQBB1PaWMpDOuJ4N7mbacOVuD6BZvj-B14I5bKF3P4xhrt6fPHFSrFBOIeNN4715cEbToWRECILbPRZzFzHMzpHLFtUCidHS-qOphkIkrG6CzYPMsxAYIcPGCQPOVvKfUQhyfKfqz4c0aQ


In [ ]:
access_token = sp.auth_manager.get_access_token(code=auth_code, check_cache=False)
print(access_token)

In [ ]:
sp.auth_manager.get_cached_token()

In [ ]:
sp.auth_manager.refresh_access_token(sp.auth_manager.get_cached_token().get('refresh_token'))

In [147]:
# r = sp.audio_features(likedSongURI.split(':')[-1])
sp.current_user()

{'display_name': 'EZWALK',
 'external_urls': {'spotify': 'https://open.spotify.com/user/wakerxd'},
 'followers': {'href': None, 'total': 18},
 'href': 'https://api.spotify.com/v1/users/wakerxd',
 'id': 'wakerxd',
 'images': [],
 'type': 'user',
 'uri': 'spotify:user:wakerxd'}

In [148]:
playlists = sp.user_playlists('wakerxd')

In [149]:
# sp.user_playlist_create('wakerxd', 'TRY~THIS', description="Auto filled using dark magic.")

In [150]:
playlists['items']#[0]['id'] # ID for TRY~THIS = 3deINaMtS15845wMyB7van
Dailys = []

# Here is a little loop to go through all of my playlists and if the name contains Daily Mix, add the URI to Dailys
# Not going to use this...
for p in playlists['items']:
    print(p['name'])
    if 'Daily Mix' in p['name']:
        Dailys.append(p['href'])

My Playlist #107
Soul♨️ofthe🌅
I found you. 
Anything but COUNTRY
LakeBand-Zürich
My Supergroup Mix
Chill Barca Chill
Walking in the rain
therapy music
Folk🎤🕴🪕alists
Summer ☀️ 22
New age ski jumps
Best show
[U+1F4F7] Whats This
An emperor’s new grüv 
Austin Roadhouse 🛣🏠
[[snaped]] EZH
🏘🥶🥶🏘
[[snaped]] grooves inside
[[snaped]] Return to me
ALT & POGL
🤪🐸
⚜️🎻⚜️
Feelings from the ♦️5️⃣0️⃣📛
Slow down sundays
👁‍🗨Lost it ⚫️
Growing up
🧟‍♂️9/12🐘
🫀GRÜVY 😵‍💫
🤘🏡🤘
Love Funk
‼️🕺🏻🔥💃‼️
[[snaped]] electroFUNK
[[snaped]] white boy summer
🗿-❄️-Chillin
[[snaped]] ski day
Lectro Chill
[[snaped]] Monkey_Music
Listen Folks
A-Town Sounds
Settin Sail
[[snaped]] kai
[[snaped]] ALTD
<movin> on
NARP
[[snaped]] wookWakeup
[[snaped]] inLoveWthis
[[snaped]] Blunts+Margs
[[snaped]] SuMMPerp
[[snaped]] goodDrivin


In [151]:
daily1Tracks#['tracks']['items']#[0]#['track']['external_urls']['spotify']
# len(contents)

NameError: name 'daily1Tracks' is not defined

In [152]:
# https://open.spotify.com/playlist/37i9dQZF1E37JNnK3FvjlV?si=m-9df_-HQlqrYHtfvx3NXA
contents = []
daily1Tracks = sp.playlist('37i9dQZF1E37JNnK3FvjlV', fields=['tracks'])
for t in daily1Tracks['tracks']['items']:
    contents.append(t['track']['href'])
    
# contents is now a list of track URIs



In [153]:
# sp.user_playlist_add_tracks('wakerxd', '3deINaMtS15845wMyB7van', songs)
daily1 = sp.playlist('37i9dQZF1E37JNnK3FvjlV')
daily1['name']

'Daily Mix 1'

In [154]:
def select_songs(songInfo): # Takes in an artist URI and the nth top played song to return. returns the URI of the artist's nth most played song
    topTracks = sp.artist_top_tracks(songInfo['artistsURIs'][0])
#     print(topTracks['tracks'][0]['name'])
    
    cSong = songInfo['uri']
    songs = set()
    if cSong != topTracks['tracks'][0]['uri']: # if the artist's most played track is not the same as the target song:
        songs.add(topTracks['tracks'][0]['uri']) # we add the artist's top song to our list to be returned
        print('Added {} to [songs]'.format(topTracks['tracks'][0]['name']))
    else:
        topTracks['tracks'][1]['uri'] # otherwise we add the second most played song
        print('Added {} to [songs]'.format(topTracks['tracks'][1]['name']))

    # we want to go through the rest of the list and if we find a song on the same album, add it to songs
    for s in topTracks['tracks']:
        if s['album']['name'] == songInfo['albumName']: # We found a song in the artist's top10 that is in the same album
            if s['name'] == songInfo['name']:
                continue
            else:
                songs.add(s['uri'])
                print('Added {} to [songs]'.format(s['name']))
                break

    print('[songs] contains {} songs'.format(songs))
    return songs

In [156]:
def get_current_info(song): # takes the response from current_user_playing_track()
    relevant_info = {}
    
    name = song['item']['name']
    relevant_info['name'] = name
    uri = song['item']['uri']
    relevant_info['uri'] = uri
#     print('Song name:\t\t', name)
#     print('Song uri:\t\t', uri)
#     print()
    
    artists = []
    artistURIs = []
    # loop thru artists and get some info
    for a in song['item']['artists']:
        artists.append(a['name'])
        artistURIs.append(a['uri'])
    #
#     print('Artists:\t\t', ', '.join(artists))
    relevant_info['artists'] = artists
#     print('Artist URIs:\t\t', ', '.join(artistURIs))
    relevant_info['artistsURIs'] = artistURIs
    
    albumType = song['item']['album']['album_type']
    relevant_info['albumType'] = albumType
    albumName = song['item']['album']['name']
    relevant_info['albumName'] = albumName
#     print('Album Name: ', albumName)
    albumURI = song['item']['album']['uri']
    relevant_info['albumURI'] = albumURI
    
    return relevant_info
    
    
#     print('Listening to "{}" by {}'.format(name, ', '.join(artists)))

In [157]:
current_song = sp.current_user_playing_track()
print('Song: ', current_song['item']['name'])
# currentSongArtistURI = current_song['item']['artists'][0]['uri']
# currentSongAlbum = current_song['item']['album']['name']
songInfo = get_current_info(current_song)
print('From album: {}'.format(songInfo['albumName']))
print('From Artist: {}'.format(songInfo['artists']))
songs = select_songs(songInfo)

Song:  Cascade
From album: Cascade
From Artist: ['Dreamers Delight']
Added The Journey to [songs]
[songs] contains {'spotify:track:5h8oWLRWiKep2Nh3s7TJnV'} songs


In [158]:
artistTop = sp.artist_top_tracks(currentSongArtistURI)

NameError: name 'currentSongArtistURI' is not defined

In [176]:
for t in artistTop['tracks']:
    if t['album']['name'] == songInfo['albumName']:
        print('true')

In [ ]:
import datetime as dt
dt.date.today().strftime()

In [178]:
songInfo = get_current_info(current_song)
# print(current_song['item']['album']['album_type']) # Single or not

Song name:		 Wonk Bros
Song uri:		 spotify:track:1QK20CJVgUSJ0ZOdfT0CKR

Artists:		 Shkspr, Olrec
Artist URIs:		 spotify:artist:2xj3fCtibuL8xKPYpSnYp4, spotify:artist:1985rOdKKZLrdPHcUIfZGY
Album Name:  Loud Speaker, Vol. 2


In [ ]:
def select_similar(songInfo):
    similar = []
    
    if album_type != 'single':
        # get the highest played song from the same album as currentSong
        # Get the album's top Track if it is not a single
        similar.append(get_albums_top_track(songInfo))
        
    # Get the artist's highest played song
    similar.append(get_artist_top_tracks(songInfo))
    
    
    
    

In [ ]:
def get_artist_top_tracks(artist_uri, nth): # Takes in an artist URI and the nth top played song to return. returns the URI of the artist's nth most played song
    topTracks = sp.artist_top_tracks(artist_uri)
    print(topTracks['tracks'][nth-1]['name'])
    
    return topTracks['tracks'][nth-1]['uri']

def get_albums_top_track(songInfo):
    tracks = sp.album_tracks(songInfo['albumURI'])
    
    albumURIs = []
    for t in tracks:
        albumURIs.append(t['uri'])
    
    
    
    return topTrack

In [152]:
tracks = sp.album_tracks(songInfo['albumURI'])
tracks.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [161]:
sp.audio_analysis(tracks['items'][0]['uri'])

{'meta': {'analyzer_version': '4.0.0',
  'platform': 'Linux',
  'detailed_status': 'OK',
  'status_code': 0,
  'timestamp': 1588914944,
  'analysis_time': 61.35847,
  'input_process': 'libvorbisfile L+R 44100->22050'},
 'track': {'num_samples': 5556600,
  'duration': 252.0,
  'sample_md5': '',
  'offset_seconds': 0,
  'window_seconds': 0,
  'analysis_sample_rate': 22050,
  'analysis_channels': 1,
  'end_of_fade_in': 0.0,
  'start_of_fade_out': 244.62222,
  'loudness': -8.388,
  'tempo': 140.006,
  'tempo_confidence': 0.647,
  'time_signature': 4,
  'time_signature_confidence': 1.0,
  'key': 5,
  'key_confidence': 0.711,
  'mode': 0,
  'mode_confidence': 0.758,
  'codestring': 'eJxdm1mC7KgORLeSS2Aw0_431ucErqq8_fH6XSjbgIZQSCLXrnWXVsqnfPoz1vP0NT5tzM_gP-tpdX5WeT7z7D2fM89n98bM6aus9amFJ8_q9Vl85VPbs53bZ_bR-qfV5-HPT61trvZpD-swLO3sclylfNapY63BEu0s1lqsWuben94my65-nt52-_R-_MCurZXdP33u8ml7jL4Hz-6HL_EE_-Otx_37Klvs5_OM2hzuh5nxeXKeDPfTPhyqfOZurbdnlM_o7OLZq5UxClIYc_vwnGxzf8bZP-9Ot1d2TtAYD4Z1RRqbBSc7-D

### Catalog functions in a dictionary and call them
 Going to be used for different argv options

In [53]:
def mult(a,b):
    return a * b

In [56]:
l = {'multiply':mult}

In [57]:
l['multiply'](1,4)

4

# Saving Daily mixes to their own playlist

In [161]:
# this should start by getting the playlist's track's URIs with the endpoint below and saving that list of track_uris
# GET https://api.spotify.com/v1/playlists/{playlist_id}/tracks
# https://open.spotify.com/playlist/37i9dQZF1E37JNnK3FvjlV?si=m-9df_-HQlqrYHtfvx3NXA
contents = []
daily1Tracks = sp.playlist('37i9dQZF1E37JNnK3FvjlV', fields=['tracks'])
for t in daily1Tracks['tracks']['items']:
    contents.append(t['track']['external_urls']['spotify'])
# contents is now a populated list of 50 track URIs. Done

# Create a new playlist named SavedDaily{N} or something like that. Use this endpoint to create a playlist
# POST https://api.spotify.com/v1/users/{user_id}/playlists
import datetime as dt
g = 1
newSavedName = 'DailySaved{}'.format(g)
creationResp = sp.user_playlist_create('wakerXD', newSavedName, description='The algorithmic vibe from {}'.format(dt.date.today()))
savedDailyID = creationResp['id']

# Next it should take this list of tracks found in the targeted DailyMix and add them to SavedDaily{N} with this endpoint
# POST https://api.spotify.com/v1/playlists/{playlist_id}/tracks
# This takes in a list of URIs to add
sp.user_playlist_add_tracks('wakerXD', savedDailyID, contents)





{'snapshot_id': 'Myw1YWRiNWVlZjk3MzU3NTA1NjYzMGQ1M2ZkY2E0ODY5M2U4MDcyNDUx'}

In [48]:
url = 'https://open.spotify.com/playlist/37i9dQZF1E35RuOD6V6IYz?si=CVAz5oL3Rd6TnlebgvLzbA'
url.split('/')[-1].split('?')[0]

'37i9dQZF1E35RuOD6V6IYz'